In [1]:
import zipfile
from os.path import basename, splitext
from pathlib import Path
from tempfile import TemporaryFile

import matplotlib.pyplot as plt
import numpy as np

projs_dir = Path() / "blueprints" / "new" / "projs"
channels_3456_dir = Path() / "blueprints" / "new" / "npy"
channels_7_dir = Path() / "blueprints" / "new" / "razryv"
channels_10_dir = Path() / "blueprints" / "new" / "razryv_area"
results_dir = Path() / "blueprints" / "new" / "mask.zip"

In [2]:
def fname(path):
    return splitext(basename(path))[0]

with zipfile.ZipFile(results_dir, 'w', zipfile.ZIP_DEFLATED) as mask_file:
    length = len(list(projs_dir.glob('*')))

    for i, proj in enumerate(sorted(projs_dir.glob('*'))):
        # names = list(map(fname, [proj, ch_3456, ch_7, ch_3456]))
        #
        # if not names[0] == names[1] == names[2] == names[3]:
        #     raise ValueError('wtf bro', names)

        ch_3456 = channels_3456_dir / f'{fname(proj)}.npy'
        ch_7 = channels_7_dir / f'{fname(proj)}.npy'
        ch_10 = channels_10_dir / f'{fname(proj)}.npy'

        name = proj
        ch_3456 = np.load(ch_3456) if ch_3456.exists() else ch_3456
        ch_7 = np.load(ch_7) if ch_7.exists() else ch_7
        ch_10 = np.load(ch_10) if ch_10.exists() else ch_10

        placeholder = np.zeros_like(ch_3456 if not isinstance(ch_3456, str) else ch_7 if not isinstance(ch_7, str) else ch_10)
        if isinstance(ch_3456, Path):
            ch_3456 = placeholder
        if isinstance(ch_7, Path):
            ch_7 = placeholder
        if isinstance(ch_10, Path):
            ch_10 = placeholder

        ch_3456 = np.stack([(ch_3456 == i).astype(float) for i in range(3, 7)])

        # print(np.max(ch_3456))

        # for ch in ch_3456:
        # fig, ax = plt.subplots(1, 6)
        # ax[0].imshow(ch_3456[0])
        # ax[1].imshow(ch_3456[1])
        # ax[2].imshow(ch_3456[2])
        # ax[3].imshow(ch_3456[3])
        # ax[4].imshow(ch_7)
        # ax[5].imshow(ch_10)
        # plt.show()

        res = np.stack([
            ch_3456[0],
            ch_3456[1],
            ch_3456[2],
            ch_3456[3],
            ch_7,
            ch_10
        ])

        # print(res.shape)

        with TemporaryFile() as numpy_temp:
            np.save(numpy_temp, res)
            numpy_temp.seek(0)
            mask_file.writestr(f'mask/{fname(name)}.npy', numpy_temp.read())

        # if i == 5:
        #     break
        print(f'{i}/{length}', end='\r')